<a href="https://colab.research.google.com/github/Issamara/RiesgosFinancieros/blob/master/Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 644 (delta 48), reused 0 (delta 0), pack-reused 579
Receiving objects: 100% (644/644), 33.00 MiB | 12.53 MiB/s, done.
Resolving deltas: 100% (427/427), done.


##Instalación de las paqueterías necesarias.


In [ ]:
%%R

install.packages("quantmod")
library(quantmod)

install.packages("data.table")
library(data.table)

install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.17.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 152204 bytes (148 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

##Funciones extras.


In [ ]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 




# Definición de parámetros para la valoración de:

Acciones, divisas, bonos, futuros, swaps y opciones europeas.

Cargamos parámetros para valoración:

In [ ]:
%%R
 
 #DATOS GENERALES

fval=as.Date("2020-02-28") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
#setwd(direc)	

#ACCIONES Y DIVISAS

#Cargar los símbolos de yahoo finance para EQ
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_eq=c(-5000,1000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_fx=c(700,-600,1500) #monto inicial invertido en divisas
nh=3660 #días de historia

#CETES

base_cetes="RiesgosFinancieros/2021-1/Tarea/tasa_guber.txt"
plazos_cetes =180 
contratos_cetes =1500

#BONO M
base_BM="RiesgosFinancieros/2021-1/Tarea/tasa_yield.txt"
tfcupon_BM =0.025 #tasa del cupon
tf_BM = 0.065 #tasa fija
plazo_BM = cbind(3600) #vencimiento del bono
plazocupon_MB =cbind( 360) #plazo fijo par cupon
contratos_BM =1
nominal = 100
itpl = 1 #0 para interpolación lineal 1 para alambrada

#BONDES D
base="RiesgosFinancieros/2021-1/Tarea/tasa_guber.txt"
btasadescst="RiesgosFinancieros/2021-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2021-1/Tarea/tfondeo.txt"
plazos_bdm=cbind(707) #Vencimiento del bono ( 358,	405,	550,	1200,	1800)
plazocupon_bdm=cbind(28) #plazos_bdm fijos de cada cupón ( 28,	28, 28, 28, 28)
contratos_bdm=cbind(-1000) #posición invertida (220, -290, 290, -460, 100)
nominal_bdm=100 #El valor nominal de estos siempre es $100

#FUTURO DE TIPO DE CAMBIO

bext="RiesgosFinancieros/2021-1/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2021-1/Tarea/tasa_fwd.txt"
btsp="RiesgosFinancieros/2021-1/Tarea/tasa_spot.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) 
plazos_fwd=cbind( 5	)
contratos_fwd=cbind(100)
kst_fwd=cbind(19.83)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FUTURO DEL IPC

#Descontamos con gubernamental
#base="RiesgosFinancieros/2021-1/Tarea/tasa_guber.txt" ESTA YA SE CARGÓ EN CETES
base2="RiesgosFinancieros/2021-1/Tarea/tasa_dividendos.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind(53) #En días
contratos_fwd_ind=cbind(50) 
kst_fwd_ind=cbind(49525) #Strike en pesos
nominal_fwd_ind=1 
yext_i=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsEQ, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt

#SWAP'S

btasadesc_sw="RiesgosFinancieros/2021-1/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2021-1/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.052,0.049) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(-1600,1200) #se establece el número de contratos_sw de cada swap (-1600,1200)
nominal_sw=cbind(1,1) #se establece el nominal_sw de cada swap (1,1)
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable (0,0)

#OPCIONES EUROPEAS

btasadesc_oir="RiesgosFinancieros/2021-1/Tarea/tasa_TIIE_SW_OP.txt" 
btasaspot_oir="RiesgosFinancieros/2021-1/Tarea/tasa_DIRS_SW_OP.txt" 
bvolspot_oir="RiesgosFinancieros/2021-1/Tarea/tvoltiie_opc.txt"
plazos_oir=cbind(1700, 700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor) (1,0,0,0,0)
K_oir=cbind( 0.094, 0.089) #( 0.07, 0.085, 0.078, 0.085, 0.08)
contratos_oir=cbind(1000, 500) #Número de contratos (2200, -2900, 2900, -460, 5000)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple



# Carga de datos

In [ ]:
%%R
btasadesc_oir="RiesgosFinancieros/2021-1/Tarea/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2021-1/Tarea/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2021-1/Tarea/tvoltiie_opc.txt"
data1<-read.table(btasadesc_oir) 





In [ ]:
%%R
#Cargar datos de BONO M
data_BM<-read.table(base_BM)
n_BM<-nrow(data_BM)
m_orig_BM=ncol(data_BM)
x_orig_BM=as.data.table(mutate(data_BM[2:n_BM,1:m_orig_BM],V1=as.Date(V1,format="%Y%m%d")))
names(x_orig_BM)[1] ="Date"
nodos_BM=data.frame(data_BM[1,2:m_orig_BM])
n_BM=n_BM-1


In [ ]:
%%R
#CARGA DE DATOS DE ACCIONES Y DIVISAS
pos=cbind(t(pos_fx),t(pos_eq))
start_date=as.Date("2020-02-28")-nh #fecha inicial
#Creación del objeto para guardar los datos
dataEnv<-new.env()
dataEnvFX<-new.env()
#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date, to=as.Date("2020-02-28"))
getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date, to=as.Date("2020-02-28"))
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(dataEnv,align='remove.na',fill.gaps = T)
bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)

#Nos quedamos con los precio
stock_prices = dataEnv$prices
stock_pricesFX=dataEnvFX$prices
#cambiar todo a pesos mexicanos
stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")
#Preciso actuales
x0=stock_prices_EQFX[nrow(stock_prices_EQFX),]

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))#para tener fechas de divisas y acciones

#LEER DATOS DE CETES #CARGA DE DATOS DE BONO CUPÓN CERO
data<-read.table(base_cetes)
n_c<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n_c,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])



##CARGA DE DATOS DE BONDE D
#carga de datos
#carga de tasas de descuento
data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
##X_orig_bd=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
#X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
#nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)
data3
X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
print(head(X2_orig_bd))
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
print(head(X2_orig_bd))
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.

print(head(X1_orig[,1:4]))


  
#CARGA DE DATOS PARA SWAP
data1<-read.table(btasadesc_sw)
n1<-nrow(data1)
m1_orig_sw=ncol(data1)
X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X1_orig_sw=X1_orig_sw%>%select(-V1)
nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
data2<-read.table(btasacupvar_sw)
n2<-nrow(data2)
m2_orig_sw=ncol(data2)
X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X2_orig_sw=X2_orig_sw%>%select(-V1)
nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  
  
#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
#carga de datos
#carga de rho	
data1<-read.table(btasadesc_oir) 
n<-nrow(data1)
m1_orig_oir=ncol(data1)
x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
x1_orig_oir=x1_orig_oir%>%select(-V1)
nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
#carga de tasas spot
data2<-read.table(btasaspot_oir)
n<-nrow(data2)
m2_orig_oir=ncol(data2)
x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
x2_orig_oir=x2_orig_oir%>%select(-V1)
nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
#carga de volatilidades de spot
data3<-read.table(bvolspot_oir)
n<-nrow(data3)
m3_orig_oir=ncol(data3)
x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
x3_orig_oir =x3_orig_oir[-c(146:148),]
x3_orig_oir=x3_orig_oir%>%select(-V1)
  
nodos3_oir=data.frame(data3[1,2:m3_orig_oir])




          V1         V2         V3         V4
1 2020-03-06 0.07906000 0.07808887 0.07709264
2 2020-03-05 0.07722139 0.07808887 0.07709264
3 2020-03-04 0.07722139 0.07808887 0.07744709
4 2020-03-03 0.07813334 0.07808887 0.07744709
5 2020-03-02 0.07722139 0.07808887 0.07744709
6 2020-02-28 0.07722139 0.07802004 0.07744709
        V1    V2
2 19981103 30.75
3 19981104  29.2
4 19981105  29.8
5 19981106  31.3
6 19981109  32.9
7 19981110 34.54
           V1   V2       Date   ID      fecha
1: 2020-03-13 7.06 2020-03-13 7802 2020-03-13
2: 2020-03-12 7.10 2020-03-12 7801 2020-03-12
3: 2020-03-11 7.02 2020-03-11 7800 2020-03-11
4: 2020-03-10 6.98 2020-03-10 7799 2020-03-10
5: 2020-03-09 7.05 2020-03-09 7798 2020-03-09
6: 2020-03-06 7.04 2020-03-08 7797 2020-03-08
           V1         V2         V3         V4
1: 2020-03-06 0.07906000 0.07808887 0.07709264
2: 2020-03-05 0.07722139 0.07808887 0.07709264
3: 2020-03-04 0.07722139 0.07808887 0.07744709
4: 2020-03-03 0.07813334 0.07808887 0.07744709
5:

In [ ]:
%%R

#CARGA DE DATOS DE FUTUROS DE TDC
data1<-read.table(bext) #Tasa libor
data2<-read.table(bdom) #tasa forward

######## minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1

###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])

####MATRICES DEL MISMO TAMAÑO MENOS DOLAR
x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)

###Para Dolar
if (yext==1)
{
  #Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvFX<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
  
  #muestra de datos
  head(dataEnvFX$prices[,2])
  
  #Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX$prices[,2])),coredata(dataEnvFX$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
#print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_ftdc=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}

# CARGA DE DATOS DE FUTUROS DE IPC
data<-read.table(base) #tasa guber
data_1<-read.table(base2) #tasa dividendos

######## minimos para parametrizar
n1=nrow(data)
n2=nrow(data_1)
m_gov=ncol(data)
m2_div=ncol(data_1)
n=min(n1,n2)-1

###NODOS###
nodos_gov=data.frame(data[1,2:m_gov])
nodos2_div=data.frame(data_1[1,2:m2_div])

####MATRICES DEL MISMO TAMAÑO MENOS IPC
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
x2_fdiv=as.data.table(mutate(data_1[2:n,1:m2_div],Date=as.Date(V1,format="%Y%m%d")))
x2_fdiv=x2_fdiv%>%select(-V1)

###Para IPC
if (yext==1)
{
  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))
}else{
# CARGA DE DATOS DE FUTUROS DE IPC
data3<-read.table(btsp)
#print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# Integración de Insumos

Nos aseguramos de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [ ]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
#prueba = intersect(x_orig_gov$Date,X3_orig_bd$Date)
#print(as.Date(prueba))
lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
lin_gub_bmybdst=data.table(Date=as.Date(lin_gub[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
#print(lin_gub_bmybdst)
lin_gub_bmybdst1=data.table(Date=as.Date(lin_gub_bmybdst[X3_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst1[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd
lin_gub_bmybdst_flibfwdspind1=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index
lin_gub_bmybdst_flibfwdspind2=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind1[x_orig_BM,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity, index or yield
lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind2[x2_fdiv,on=.(Date),nomatch=0]$Date))
###################NOTA
#Hasta este momento se unieron las bases necesarias para las acciones, divisas, futuros y además una base para bondes y bonos

lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
#print(nrow(lin_gub_bmybdst_flibfwdspind_swcup))
lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
#print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp))
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot
#print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs))
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
#print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp))

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol
#print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol))

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
#print(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
print(nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol))

###lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol sera la base con TODOS los insumos necesarios, 
#aunque tengan que agregar más, hagan que la final sea esa para no tener problemas después 

[1] 151


In [ ]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos

#Historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#Historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)
print(dim(x_orig_gov))

#Historia de curva yield
x_orig_BM =lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_BM,on=.(Date),nomatch=0][order(-Date)]
x_orig_BM=x_orig_BM%>%select(-Date)

#Historia de curvas de bondes
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X1_orig=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)
# X2_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
# X2_orig_bd=X2_orig_bd%>%select(-Date)
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)

#Historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)


#Historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL x_ORIG_GOV
x2_fdiv=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_fdiv,on=.(Date),nomatch=0][order(-Date)]
x2_fdiv=x2_fdiv%>%select(-Date)
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#Historia de swaps
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#Historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)



[1] 154  20


# Pasos para medir el riesgo por el método de Simulación Histórica

Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Valoración al día actual

## Acciones y Divisas



In [ ]:
%%R
#divisas y acciones CÁLCULO

x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
m_fx=length(pos_fx)
m_acc=length(pos_eq)
x0_acc_div
print(V0_acc_div)

   EURUSD.X GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1: 14696.63 -14935.8  28929.9  -77350       56720     66408


## CETES

In [ ]:
%%R
#Cete CÁLCULO
plazos_cetes =cbind(180)
m=ncol(plazos_cetes) #solo hay uno en este caso 
x_cete=matrix(0,n,m)


for (i in 1:(n-1))
{

x_cete[i,]=approx(nodos_gov,x_orig_gov[i,],plazos_cetes)$y

}

x0_cete=x_cete[1,]
x0_cete #valor actual
#función
bonocupcero = function(i,t)
{
1/(1+i*t/360)
}
nominal =10
V0_cete=as.matrix(bonocupcero(x0_cete,plazos_cetes))*contratos_cetes*nominal #Valor actual de cada bono
VT0_cete=sum(V0_cete) #Valor total del portafolio al tiempo 0
VT0_cete
#en este caso como solo tenemos uno deben ser los mismos
#x0_cete

[1] 14416.31


## Bono M

In [ ]:
%%R
#Calculo Bono M
#intersección de tasas y volatilidades
m=ncol(plazo_BM)
X_BM=matrix(0,n-1,m)
itpl=0
for (i in 1:(n-1))
{
X_BM[i,]=if(itpl==0){approx(nodos_BM,x_orig_BM[i,],plazo_BM)$y}else{talamb(nodos_BM,x_orig_BM[i,],plazo_BM)}
}
#Función de valoración por tasa yield
bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
x=x0  
N=as.integer(plazos/plazocupon)+1   
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}
plazocupon_BM= 0.025
x0=X_BM[1,] #tasas de descuento valor actual
V0_BM =bonoMyield(x0,plazo_BM, plazocupon_BM, tfcupon_BM, 100, contratos_BM)
#print(x_orig_BM)
V0_BM

         [,1]
[1,] 63.06883


## Bondes D

In [ ]:
%%R
#BONDE D CÁLCULO

X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)

V0_bd



          [,1]
[1,] -95226.61


## Futuros de Tipo de Cambio

In [ ]:

%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO
################MATRICES DE INTERPOLACION LINEAL ####################
m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)
 
for (j in 1:n){
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}
 
futuroTC = function(t,tl,tn,s,k){ #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}
 
X3_ftdc=as.matrix(X3_ftdc)
X_futtdc=cbind(X1_fwtdc,X2_fwtdc,X3_ftdc)
 
V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd
print(V0_fwtdc)

          [,1]
[1,] -53.23399


## Futuros de IPC

In [ ]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################
m_ind=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m_ind) #DIVIDENDOS
X2_fwind=matrix(0,n,m_ind)
 
for (j in 1:n){
  #X1_fwind[j,]=if(itpl==0){approx(nodos2_div,x2_fdiv[j,],plazos_fwd_ind, rule=2)$y}else{talamb(nodos2_div,x2_fdiv[j,],plazos_fwd_ind)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
}

futuroI = function(t,td,tn,s,k){ #t=dias por vencer, tn=tasa nacional para indice forward, td= tasa dividendos forward, S=spot
  f=s*(1+(tn-td)*t/360) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_find=as.matrix(X3_find)
X_futind=cbind(X1_fwind,X2_fwind,matrix(X3_find,n,ncol(X1_fwind)))
 
V0_fwind=futuroI(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
print(V0_fwind)

          [,1]
[1,] -367656.2


Total para los futuros:

In [ ]:
%%R
V0_port=cbind(V0_fwtdc, V0_fwind)
V0T_port=sum(V0_port)
V0T_port

[1] -367709.4


## SWAPS

In [ ]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)
V0_sw

          [,1]      [,2]
[1,] -63.75999 -28.39141


##Opciones de Tasa de Interés

In [ ]:
%%R

##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)

for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
V0_oir

         [,1]     [,2]
[1,] 2.215311 0.121693


# Integración de factores y cálculo de riesgo en conjunto, y aplicación de simulación

In [ ]:
%%R
V0_port=cbind(V0_acc_div,V0_bd, V0_fwtdc,V0_fwind,V0_sw,V0_oir,V0_cete, V0_BM)
V0T_port=sum(V0_port)
V0_port

   EURUSD.X GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX        V1        V1
1: 14696.63 -14935.8  28929.9  -77350       56720     66408 -95226.61 -53.23399
          V1        V1        V2       V1       V2       V1       V1
1: -367656.2 -63.75999 -28.39141 2.215311 0.121693 14416.31 63.06883


In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,8,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(X_bd_ext) #bonde
n_if[3]=ncol(X_sw) #swaps
n_if[4]=ncol(X_oir) #opciones tasa de interés
n_if[5]=ncol(X_futtdc) #Forwards de tipo de cambio
n_if[6]=ncol(X_futind) #Forwards de índices
n_if[7]=ncol(x_cete)   #cete
n_if[8]=ncol(X_BM)   #bono M
#valor del portafolios

V0_port=cbind(V0_acc_div, V0_bd, V0_sw, V0_oir, V0_fwtdc,V0_fwind,V0_cete, V0_BM)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,X_bd_ext,X_sw,X_oir,X_futtdc,X_futind,x_cete,X_BM) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[2:(n)]/X_port[1:(n-1)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
#alpha=0.98 #Nivel de Confianza para las medidas de riesgo

DeltaX_s=DeltaX_port
#print(head(DeltaX_s))
print(ncol(DeltaX_s))
print(nrow(DeltaX_s))
print(n_if)


[1] 162
[1] 150
     [,1]
[1,]    6
[2,]   78
[3,]   64
[4,]    6
[5,]    3
[6,]    3
[7,]    1
[8,]    1


In [ ]:
%%R

#Medidas de riesgo CON alisado

#Se necesita definir
#1) El valor del peso inicial del primer escenario "w0"
#2) La función de cuantil con vector de probabilidades no iguales
#3) La función de CVaR con probabilidades no iguales

#w0=0.05

#Creación de dos funciones que sirven para este fin
# Percentil con pesos de probabilidades
# 
# v  un vector de observaciones
# w Un vector numérico de valores positivos, en general es la distrubición.
# p  el valor de la probabilidad entre 0 y 1.
# 
# Esta función no interpola

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 

w0=0.05
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root
lambda

#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}


p_esc=genera_esc(lambda,w0,Ns)
print(t(p_esc[1:10]))
sum(p_esc) #validamos que sume 1

     [,1]       [,2]       [,3]       [,4]       [,5]       [,6]       [,7]
[1,] 0.05 0.04750109 0.04512706 0.04287169 0.04072904 0.03869347 0.03675964
           [,8]       [,9]      [,10]
[1,] 0.03492245 0.03317709 0.03151895
[1] 0.9999771


# Medición de Riesgo

## Acciones y divisas

In [ ]:
%%R
#Medición de riesgo por instrumento, instrumento-factor de riesgo, instrumento - total
#Cálculo de matriz de pérdidas y ganancias Acciones y Divisas
#riesgo del acciones y divisas
m=m_fx+m_acc    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_fx)
Vfr2_acc_div=matrix(0,Ns,m_acc)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_fx)
PGfr2_acc_div=matrix(0,Ns,m_acc)
PGT_acc_div=matrix(0,Ns,1)
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

DeltaX_s_acc_div=DeltaX_s[,(1:n_if[1])]    #PASO CLAVE
x0_acc_div=stock_prices_EQFX[1,]   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*exp(DeltaX_s_acc_div[i,]))
  #PASO CLAVE
  V_acc_div[i,]=cbind(t(pos_fx),t(pos_eq))*X_s_acc_div[i,]
  #PASO CLAVE
  Vfr1_acc_div[i,]=t(pos_fx)*X_s_acc_div[i,1:m_fx]
  #PASO CLAVE
  Vfr2_acc_div[i,]=t(pos_eq)*X_s_acc_div[i,(m_fx+1):(m_fx+m_acc)]
  #PASO CLAVE
  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)
  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_fx])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_fx+1):(m_fx+m_acc)])
  PGT_acc_div[i,]=sum(PG_acc_div[i,])
  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}

VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_fx)
VaRfr2_acc_div=matrix(0,1,m_acc)
CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_fx)
CVaRfr2_acc_div=matrix(0,1,m_acc)

VaRCont_CA_acc_div=matrix(0,1,m)
VaRfr1_CA_acc_div=matrix(0,1,m_fx)
VaRfr2_CA_acc_div=matrix(0,1,m_acc)
CVaRCont_CA_acc_div=matrix(0,1,m)
CVaRfr1_CA_acc_div=matrix(0,1,m_fx)
CVaRfr2_CA_acc_div=matrix(0,1,m_acc)

#VaR por posición CON y SIN alisado
for (i in (1:m)){
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  VaRCont_CA_acc_div[i]=wquantile(PG_acc_div[,i],p_esc, 1-alpha)
  CVaRCont_CA_acc_div[i]= wcvar(PG_acc_div[,i],p_esc, 1-alpha)
  if (i<=m_fx){
  VaRfr1_acc_div[i]=equantile(PGfr1_acc_div[,i],1-alpha,Ns)
  CVaRfr1_acc_div[i]= mean(merge(which(PGfr1_acc_div[,i]<VaRfr1_acc_div[i]),cbind(seq(1,Ns),PGfr1_acc_div[,i]), by.x=1,by.y=1)[,2])
  VaRfr1_CA_acc_div[i]=wquantile(PGfr1_acc_div[,i], p_esc, 1-alpha)
  CVaRfr1_CA_acc_div[i]=wcvar(PGfr1_acc_div[,i], p_esc, 1-alpha)
  }
  if (i<=m_acc){  
  VaRfr2_acc_div[i]=equantile(PGfr2_acc_div[,i],1-alpha,Ns)
  CVaRfr2_acc_div[i]= mean(merge(which(PGfr2_acc_div[,i]<VaRfr2_acc_div[i]),cbind(seq(1,Ns),PGfr2_acc_div[,i]), by.x=1,by.y=1)[,2])
  VaRfr2_CA_acc_div[i]=wquantile(PGfr2_acc_div[,i], p_esc, 1-alpha)
  CVaRfr2_CA_acc_div[i]=wcvar(PGfr2_acc_div[,i], p_esc, 1-alpha)
  }
}

#VaR Total SIN alisado
VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])

#VaR Total CON alisado 
VaRTotal_acc_divCA=wquantile(PGT_acc_div,p_esc, 1-alpha)
CVaRTotal_acc_divCA= wcvar(PGT_acc_div,p_esc, 1-alpha)
VaRTotalfr1_acc_divCA=wquantile(PGfr1T_acc_div, p_esc, 1-alpha)
CVaRTotalfr1_acc_divCA= wcvar(PGfr1T_acc_div, p_esc, 1-alpha)
VaRTotalfr2_acc_divCA=wquantile(PGfr2T_acc_div, p_esc, 1-alpha)
CVaRTotalfr2_acc_divCA= wcvar(PGfr2T_acc_div, p_esc, 1-alpha)

resultadosA<-as.data.frame(rbind(VaRfr2_acc_div, CVaRfr2_acc_div))
colnames(resultadosA)<- c("America Móvil", "Grupo Carso", "WalMart")
rownames(resultadosA)<- c("VaR SA", "CVaR SA")
print(resultadosA)

resultadosD<-as.data.frame(rbind(VaRfr1_acc_div, CVaRfr1_acc_div))
colnames(resultadosD)<- c("Euro", "Libra", "Dolar")
rownames(resultadosD)<- c("VaR SA", "CVaR SA")
print(resultadosD)

resultadosAA<-as.data.frame(rbind(VaRfr2_CA_acc_div, CVaRfr2_CA_acc_div))
colnames(resultadosAA)<- c("America Móvil", "Grupo Carso", "WalMart")
rownames(resultadosAA)<- c("VaR CA", "CVaR CA")
print(resultadosAA)

resultadosDA<-as.data.frame(rbind(VaRfr1_CA_acc_div, CVaRfr1_CA_acc_div))
colnames(resultadosDA)<- c("Euro", "Libra", "Dolar")
rownames(resultadosDA)<- c("VaR CA", "CVaR CA")
print(resultadosDA)

resultadosA1<-as.data.frame(rbind(VaRTotalfr2_acc_div, CVaRTotalfr2_acc_div))
colnames(resultadosA1)<- c("Total Acciones")
rownames(resultadosA1)<- c("VaR SA", "CVaR SA")
print(resultadosA1)

resultadosD1<-as.data.frame(rbind(VaRTotalfr1_acc_div, CVaRTotalfr1_acc_div))
colnames(resultadosD1)<- c("Total Divisas")
rownames(resultadosD1)<- c("VaR SA", "CVaR SA")
print(resultadosD1)

resultadosA2<-as.data.frame(rbind(VaRTotalfr2_acc_divCA, CVaRTotalfr2_acc_div))
colnames(resultadosA1)<- c("Total Acciones")
rownames(resultadosA1)<- c("VaR CA", "CVaR CA")
print(resultadosA1)

resultadosD2<-as.data.frame(rbind(VaRTotalfr1_acc_divCA, CVaRTotalfr1_acc_div))
colnames(resultadosD1)<- c("Total Divisas")
rownames(resultadosD1)<- c("VaR CA", "CVaR CA")
print(resultadosD1)



        America Móvil Grupo Carso   WalMart
VaR SA      -3178.767   -2813.216 -1776.159
CVaR SA     -4763.664   -3176.091 -2651.950
             Euro     Libra    Dolar
VaR SA  -227.3721 -182.1861 -436.378
CVaR SA -429.6876 -205.6910 -479.467
        America Móvil Grupo Carso   WalMart
VaR CA      -2907.890   -2796.881 -1750.915
CVaR CA     -4244.007   -3213.160 -2258.776
             Euro     Libra     Dolar
VaR CA  -202.1940 -175.0739 -456.3435
CVaR CA -313.0922 -185.9979 -502.5906
        Total Acciones
VaR SA       -3500.695
CVaR SA      -3629.971
        Total Divisas
VaR SA      -466.2193
CVaR SA     -569.8692
        Total Acciones
VaR CA       -3500.695
CVaR CA      -3629.971
        Total Divisas
VaR CA      -466.2193
CVaR CA     -569.8692


##Riesgo Cetes

In [339]:
%%R
#Para cetes solo hay un factor de riesgo la tasa gubernamental
#dimensión
m=ncol(plazos_cetes)   #PASO CLAVE
X_s_cete=matrix(0,Ns,n_if[7]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_cete=matrix(0,Ns,m)
Vfr1_cete=matrix(0,Ns,m)
PG_cete=matrix(0,Ns,m) #Pèrdidas y ganancias
PGT_cete=matrix(0,Ns,m)
PGfr1_cete=matrix(0,Ns,m)
PGfr1T_cete=matrix(0,Ns,m)
DeltaX_s_cete=as.matrix(DeltaX_s[,sum(n_if[1:6],1):sum(n_if[1:7])])  #PASO CLAVE
x0_cete=x_cete[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_cete[i,]=x0_cete*exp(DeltaX_s_cete[i,])
  #PASO CLAVE

  V_cete[i,]=as.matrix(bonocupcero(X_s_cete][i,1:((n_if[7]))],plazos_cetes))*contratos_cetes*10
  #PASO CLAVE
  Vfr1_cete[i,]=as.matrix(bonocupcero(x0_cete,plazos_cetes))*contratos_cetes*10
  #PASO CLAVE
  PG_cete[i,]=V_cete[i,]-V0_cete
  PGfr1_cete[i,]=Vfr1_cete[i,]-V0_cete
  PGT_cete[i,]=sum(PG_cete[i,])
  PGfr1T_cete[i,]=sum(PGfr1_cete[i,])
}



PG_cete[1:5,]
PGfr1_cete[1:5,]
PGT_cete[1:5,]


#VaR por posición
VaRCont_cete=matrix(0,1,m)
VaRfr1_cete=matrix(0,1,m)
CVaRCont_cete=matrix(0,1,m)
CVaRfr1_cete=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_cete[i]=equantile(PG_cete[,i],1-alpha,Ns)
  VaRfr1_cete[i]=equantile(PGfr1_cete[,i],1-alpha,Ns)
  CVaRfr1_cete[i]= mean(merge(which(PGfr1_cete[,i]<VaRfr1_cete[i]),cbind(seq(1,Ns),PGfr1_cete[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_cete[i]= mean(merge(which(PG_cete[,i]<VaRCont_cete[i]),cbind(seq(1,Ns),PG_cete[,i]), by.x=1,by.y=1)[,2])
}

VaRCont_cete
VaRfr1_cete
CVaRCont_cete
CVaRfr1_cete



#VaR Total

VaRTotal_cete=equantile(PGT_cete,1-alpha,Ns)
CVaRTotal_cete= mean(merge(which(PGT_cete<VaRTotal_cete),cbind(seq(1,Ns),PGT_cete), by.x=1,by.y=1)[,2])
VaRTotalfr1_cete=equantile(PGfr1T_cete,1-alpha,Ns)
CVaRTotalfr1_cete= mean(PGfr1T_cete[which(PGfr1T_cete<VaRTotalfr1_cete),])

print(cbind(VaRTotal_cete,sum(V0_cete), VaRCont_cete, V0_cete))
print(cbind(CVaRTotal_cete,sum(V0_cete), CVaRCont_cete, V0_cete))
#print(cbind(VaRTotal_cete,VaRTotalfr1_cete))
#print(cbind(CVaRTotal_cete,CVaRTotalfr1_cete))

RParsingError: ignored

##Riesgo Bono M

Riesgo de Bonos M

In [ ]:
%%R

#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(plazo_BM/plazocupon_BM)    #PASO CLAVE
X_s_BM=matrix(0,Ns,n_if[8]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_BM=matrix(0,Ns,m)
Vfr1_BM=matrix(0,Ns,m)
Vfr2_BM=matrix(0,Ns,m)
PG_BM=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_BM=matrix(0,Ns,m)
PGfr2_BM=matrix(0,Ns,m)
PGT_BM=matrix(0,Ns,1)
PGfr1T_BM=matrix(0,Ns,1)
PGfr2T_BM=matrix(0,Ns,1)


DeltaX_s_BM=DeltaX_s[,sum(n_if[1:7]):sum(n_if[1:8])]    #PASO CLAVE
x0_BM=X_BM[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_BM[i,]=x0_BM*exp(DeltaX_s_BM[i,])
  #PASO CLAVE
 
  V_BM[i,]=bonoMyield(x0_BM,plazo_BM, plazocupon_BM, X_s_BM[i,1], 100, contratos_BM)
  #PASO CLAVE
  Vfr1_BM[i,]=bonoMyield(x0_BM,plazo_BM, plazocupon_BM, X_s_BM[i,1], 100, contratos_BM)
  #PASO CLAVE
  Vfr2_BM[i,]=bonoMyield(x0_BM,plazo_BM, plazocupon_BM, x0_BM, 100, contratos_BM)
  #PASO CLAVE
  Vfr3_BM[i,]=bonoMyield(x0_BM,plazo_BM, plazocupon_BM, x0_BM, 100, contratos_BM)
  #PASO CLAVE
  PG_BM[i,]=V_BM[i,]-V0_BM
  PGfr1_BM[i,]=Vfr1_BM[i,]-V0_BM
  PGfr2_BM[i,]=Vfr2_BM[i,]-V0_BM

  PGT_BM[i,]=sum(PG_BM[i,])
  PGfr1T_BM[i,]=sum(PGfr1_BM[i,])
  PGfr2T_BM[i,]=sum(PGfr2_BM[i,])

}


#VaR por posición
VaRCont_BM=matrix(0,1,m)
VaRfr1_BM=matrix(0,1,m)
VaRfr2_BM=matrix(0,1,m)
CVaRCont_BM=matrix(0,1,m)
CVaRfr1_BM=matrix(0,1,m)
CVaRfr2_BM=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_BM[i]=equantile(PG_BM[,i],1-alpha,Ns)
  VaRfr1_BM[i]=equantile(PGfr1_BM[,i],1-alpha,Ns)
  VaRfr2_BM[i]=equantile(PGfr2_BM[,i],1-alpha,Ns)
  CVaRfr1_BM[i]= mean(merge(which(PGfr1_BM[,i]<VaRfr1_BM[i]),cbind(seq(1,Ns),PGfr1_BM[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_BM[i]= mean(merge(which(PGfr2_BM[,i]<VaRfr2_BM[i]),cbind(seq(1,Ns),PGfr2_BM[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_BM[i]= mean(merge(which(PG_BM[,i]<VaRCont_BM[i]),cbind(seq(1,Ns),PG_BM[,i]), by.x=1,by.y=1)[,2])
}

VaRCont_BM
VaRfr1_BM
VaRfr2_BM
CVaRCont_BM
CVaRfr1_BM
CVaRfr2_BM


#VaR Total

VaRTotal_BM=equantile(PGT_BM,1-alpha,Ns)
CVaRTotal_BM= mean(merge(which(PGT_BM<VaRTotal_BM),cbind(seq(1,Ns),PGT_BM), by.x=1,by.y=1)[,2])
VaRTotalfr1_BM=equantile(PGfr1T_BM,1-alpha,Ns)
CVaRTotalfr1_BM= mean(PGfr1T_BM[which(PGfr1T_BM<VaRTotalfr1_BM),])
VaRTotalfr2_BM=equantile(PGfr2T_BM,1-alpha,Ns)
CVaRTotalfr2_BM= mean(PGfr2T_BM[which(PGfr2T_BM<VaRTotalfr2_BM),])



print(cbind(VaRTotal_BM,sum(V0_BM), VaRCont_BM, V0_BM))
print(cbind(CVaRTotal_BM,sum(V0_BM), CVaRCont_BM, V0_BM))
print(cbind(VaRTotal_BM,VaRTotalfr1_BM,VaRTotalfr2_BM))
print(cbind(CVaRTotal_BM,CVaRTotalfr1_BM,CVaRTotalfr2_BM))


R[write to console]: Error in X_s_BM[i, ] <- x0_BM * exp(DeltaX_s_BM[i, ]) : 
  number of items to replace is not a multiple of replacement length
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in X_s_BM[i, ] <- x0_BM * exp(DeltaX_s_BM[i, ]) : 
  number of items to replace is not a multiple of replacement length
Calls: <Anonymous> -> <Anonymous> -> withVisible


## Riesgo de Bondes D

In [326]:
%%R

#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(N_bd)    #PASO CLAVE
X_s_bd=matrix(0,Ns,n_if[2]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_bd=matrix(0,Ns,m)
Vfr1_bd=matrix(0,Ns,m)
Vfr2_bd=matrix(0,Ns,m)
Vfr3_bd=matrix(0,Ns,m)
PG_bd=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_bd=matrix(0,Ns,m)
PGfr2_bd=matrix(0,Ns,m)
PGfr3_bd=matrix(0,Ns,m)
PGT_bd=matrix(0,Ns,1)
PGfr1T_bd=matrix(0,Ns,1)
PGfr2T_bd=matrix(0,Ns,1)
PGfr3T_bd=matrix(0,Ns,1)

DeltaX_s_bd=DeltaX_s[,sum(n_if[1:1],1):sum(n_if[1:2])]    #PASO CLAVE
x0_bd=X_bd_ext[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_bd[i,]=x0_bd*exp(DeltaX_s_bd[i,])
  #PASO CLAVE
  V_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr1_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr2_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr3_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  PG_bd[i,]=V_bd[i,]-V0_bd
  PGfr1_bd[i,]=Vfr1_bd[i,]-V0_bd
  PGfr2_bd[i,]=Vfr2_bd[i,]-V0_bd
  PGfr3_bd[i,]=Vfr3_bd[i,]-V0_bd
  PGT_bd[i,]=sum(PG_bd[i,])
  PGfr1T_bd[i,]=sum(PGfr1_bd[i,])
  PGfr2T_bd[i,]=sum(PGfr2_bd[i,])
  PGfr3T_bd[i,]=sum(PGfr3_bd[i,])
}



PG_bd[1:5,]
PGfr1_bd[1:5,]
PGfr2_bd[1:5,]
PGT_bd[1:5,]


#VaR por posición
VaRCont_bd=matrix(0,1,m)
VaRfr1_bd=matrix(0,1,m)
VaRfr2_bd=matrix(0,1,m)
VaRfr3_bd=matrix(0,1,m)
CVaRCont_bd=matrix(0,1,m)
CVaRfr1_bd=matrix(0,1,m)
CVaRfr2_bd=matrix(0,1,m)
CVaRfr3_bd=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_bd[i]=equantile(PG_bd[,i],1-alpha,Ns)
  VaRfr1_bd[i]=equantile(PGfr1_bd[,i],1-alpha,Ns)
  VaRfr2_bd[i]=equantile(PGfr2_bd[,i],1-alpha,Ns)
  VaRfr3_bd[i]=equantile(PGfr3_bd[,i],1-alpha,Ns)
  CVaRfr1_bd[i]= mean(merge(which(PGfr1_bd[,i]<VaRfr1_bd[i]),cbind(seq(1,Ns),PGfr1_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_bd[i]= mean(merge(which(PGfr2_bd[,i]<VaRfr2_bd[i]),cbind(seq(1,Ns),PGfr2_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_bd[i]= mean(merge(which(PGfr3_bd[,i]<VaRfr3_bd[i]),cbind(seq(1,Ns),PGfr3_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_bd[i]= mean(merge(which(PG_bd[,i]<VaRCont_bd[i]),cbind(seq(1,Ns),PG_bd[,i]), by.x=1,by.y=1)[,2])
}

VaRCont_bd
VaRfr1_bd
VaRfr2_bd
CVaRCont_bd
CVaRfr1_bd
CVaRfr2_bd


#VaR Total

VaRTotal_bd=equantile(PGT_bd,1-alpha,Ns)
CVaRTotal_bd= mean(merge(which(PGT_bd<VaRTotal_bd),cbind(seq(1,Ns),PGT_bd), by.x=1,by.y=1)[,2])
VaRTotalfr1_bd=equantile(PGfr1T_bd,1-alpha,Ns)
CVaRTotalfr1_bd= mean(PGfr1T_bd[which(PGfr1T_bd<VaRTotalfr1_bd),])
VaRTotalfr2_bd=equantile(PGfr2T_bd,1-alpha,Ns)
CVaRTotalfr2_bd= mean(PGfr2T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])
VaRTotalfr3_bd=equantile(PGfr3T_bd,1-alpha,Ns)
CVaRTotalfr3_bd= mean(PGfr3T_bd[which(PGfr2T_bd<VaRTotalfr3_bd),])


print(cbind(VaRTotal_bd,sum(V0_bd), VaRCont_bd, V0_bd))
print(cbind(CVaRTotal_bd,sum(V0_bd), CVaRCont_bd, V0_bd))
print(cbind(VaRTotal_bd,VaRTotalfr1_bd,VaRTotalfr2_bd,VaRTotalfr3_bd))
print(cbind(CVaRTotal_bd,CVaRTotalfr1_bd,CVaRTotalfr2_bd,CVaRTotalfr3_bd))


     VaRTotal_bd                             
[1,]    4213.218 -95226.61 4213.218 -95226.61
     CVaRTotal_bd                             
[1,]     4165.866 -95226.61 4165.866 -95226.61
     VaRTotal_bd VaRTotalfr1_bd VaRTotalfr2_bd VaRTotalfr3_bd
[1,]    4213.218        3980.59       4140.067       4140.067
     CVaRTotal_bd CVaRTotalfr1_bd CVaRTotalfr2_bd CVaRTotalfr3_bd
[1,]     4165.866        3828.259        4070.462        4070.462


In [ ]:
%%R
#Con alisado
VaRTotal_bd=wquantile(PGT_bd,p_esc,1-alpha)
CVaRTotal_bd= wcvar(merge(which(PGT_bd<VaRTotal_bd),cbind(seq(1,Ns),PGT_bd), by.x=1,by.y=1)[,2])
VaRTotalfr1_bd=wquantile(PGfr1T_bd,p_esc,1-alpha)
CVaRTotalfr1_bd= wcvar(PGfr1T_bd[which(PGfr1T_bd<VaRTotalfr1_bd),])
VaRTotalfr2_bd=wquantile(PGfr2T_bd,p_esc,1-alpha)
CVaRTotalfr2_bd= wcvar(PGfr2T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])
VaRTotalfr3_bd=wquantile(PGfr3T_bd,p_esc,1-alpha)
CVaRTotalfr3_bd= wcvar(PGfr3T_bd[which(PGfr2T_bd<VaRTotalfr3_bd),])

print(cbind(VaRTotal_bd,sum(V0_bd), VaRCont_bd, V0_bd))
print(cbind(CVaRTotal_bd,sum(V0_bd), CVaRCont_bd, V0_bd))
print(cbind(VaRTotal_bd,VaRTotalfr1_bd,VaRTotalfr2_bd,VaRTotalfr3_bd))
print(cbind(CVaRTotal_bd,CVaRTotalfr1_bd,CVaRTotalfr2_bd,CVaRTotalfr3_bd))

     VaRTotal_bd                             
[1,]    4250.206 -95226.61 4213.218 -95226.61
     CVaRTotal_bd                             
[1,]     4146.437 -95226.61 4165.866 -95226.61
     VaRTotal_bd VaRTotalfr1_bd VaRTotalfr2_bd VaRTotalfr3_bd
[1,]    4250.206       3996.719       4194.678       4194.678
     CVaRTotal_bd CVaRTotalfr1_bd CVaRTotalfr2_bd CVaRTotalfr3_bd
[1,]     4146.437        3728.729        4041.955        4041.955


## Riesgo SWAPS

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias SWAP

#riesgo del swap
m=ncol(N)    #PASO CLAVE
X_s_sw=matrix(0,Ns,n_if[3]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_sw=matrix(0,Ns,m)
Vfr1_sw=matrix(0,Ns,m)
Vfr2_sw=matrix(0,Ns,m)
PG_sw=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_sw=matrix(0,Ns,m)
PGfr2_sw=matrix(0,Ns,m)
PGT_sw=matrix(0,Ns,1)
PGfr1T_sw=matrix(0,Ns,1)
PGfr2T_sw=matrix(0,Ns,1)

DeltaX_s_sw=DeltaX_s[,sum(n_if[1:2],1):sum(n_if[1:3])]    #PASO CLAVE
x0_sw=as.numeric(c(XtfwdT[1,],Xvp[1,]))   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_sw[i,]=x0_sw*exp(DeltaX_s_sw[i,])
  #PASO CLAVE
  V_sw[i,]=swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  Vfr1_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,],N)
  #PASO CLAVE
  Vfr2_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  PG_sw[i,]=V_sw[i,]-V0_sw
  PGfr1_sw[i,]=Vfr1_sw[i,]-V0_sw
  PGfr2_sw[i,]=Vfr2_sw[i,]-V0_sw
  PGT_sw[i,]=sum(PG_sw[i,])
  PGfr1T_sw[i,]=sum(PGfr1_sw[i,])
  PGfr2T_sw[i,]=sum(PGfr2_sw[i,])
}

PG_sw[1:5,]
PGfr1_sw[1:5,]
PGfr2_sw[1:5,]
PGT_sw[1:5,]

#VaR por posición
VaRCont_sw=matrix(0,1,m)
VaRfr1_sw=matrix(0,1,m)
VaRfr2_sw=matrix(0,1,m)
CVaRCont_sw=matrix(0,1,m)
CVaRfr1_sw=matrix(0,1,m)
CVaRfr2_sw=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_sw[i]=equantile(PG_sw[,i],1-alpha,Ns)
  VaRfr1_sw[i]=equantile(PGfr1_sw[,i],1-alpha,Ns)
  VaRfr2_sw[i]=equantile(PGfr2_sw[,i],1-alpha,Ns)
  CVaRfr1_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])
}
VaRCont_sw
VaRfr1_sw
VaRfr2_sw
CVaRCont_sw
CVaRfr1_sw
CVaRfr2_sw


#VaR Total

VaRTotal_sw=equantile(PGT_sw,1-alpha,Ns)
CVaRTotal_sw= mean(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])
VaRTotalfr1_sw=equantile(PGfr1T_sw,1-alpha,Ns)
CVaRTotalfr1_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])
VaRTotalfr2_sw=equantile(PGfr2T_sw,1-alpha,Ns)
CVaRTotalfr2_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])


print(cbind(VaRTotal_sw,sum(V0_sw), VaRCont_sw, V0_sw))
print(cbind(CVaRTotal_sw,sum(V0_sw), CVaRCont_sw, V0_sw))
print(cbind(VaRTotal_sw,VaRTotalfr1_sw,VaRTotalfr2_sw))
print(cbind(CVaRTotal_sw,CVaRTotalfr1_sw,CVaRTotalfr2_sw))


     VaRTotal_sw                                                   
[1,]    -3.79124 -92.15139 -3.202085 -0.9217175 -63.75999 -28.39141
     CVaRTotal_sw                                                  
[1,]    -5.037673 -92.15139 -4.684069 -1.149495 -63.75999 -28.39141
     VaRTotal_sw VaRTotalfr1_sw VaRTotalfr2_sw
[1,]    -3.79124      -3.786385    -0.03494778
     CVaRTotal_sw CVaRTotalfr1_sw CVaRTotalfr2_sw
[1,]    -5.037673       -5.086561      -0.1946292


In [ ]:
%%R
#Con alisado
VaRTotal_sw=wquantile(PGT_sw,p_esc,1-alpha)
CVaRTotal_sw= wcvar(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])
VaRTotalfr1_sw=wquantile(PGfr1T_sw,p_esc,1-alpha)
CVaRTotalfr1_sw= wcvar(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])
VaRTotalfr2_sw=wquantile(PGfr2T_sw,p_esc,1-alpha)
CVaRTotalfr2_sw= wcvar(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])


print(cbind(VaRTotal_sw,sum(V0_sw), VaRCont_sw, V0_sw))
print(cbind(CVaRTotal_sw,sum(V0_sw), CVaRCont_sw, V0_sw))
print(cbind(VaRTotal_sw,VaRTotalfr1_sw,VaRTotalfr2_sw))
print(cbind(CVaRTotal_sw,CVaRTotalfr1_sw,CVaRTotalfr2_sw))

     VaRTotal_sw                                                   
[1,]   -1.730006 -92.15139 -3.202085 -0.9217175 -63.75999 -28.39141
     CVaRTotal_sw                                                  
[1,]    -4.001329 -92.15139 -4.684069 -1.149495 -63.75999 -28.39141
     VaRTotal_sw VaRTotalfr1_sw VaRTotalfr2_sw
[1,]   -1.730006      -1.721671    -0.03147409
     CVaRTotal_sw CVaRTotalfr1_sw CVaRTotalfr2_sw
[1,]    -4.001329       -4.017361             NaN


## Opciones Tasa de Interés

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias Opciones Tasa de interés
#dimensión
m=ncol(plazos_oir)  #PASO CLAVE
X_s_oir=matrix(0,Ns,n_if[4]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s[,sum(n_if[1:3],1):sum(n_if[1:4])]  #PASO CLAVE
x0_oir=X_oir[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*exp(DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_oir
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}



PG_oir[1:5,]
PGfr1_oir[1:5,]
PGfr2_oir[1:5,]
PGT_oir[1:5,]


#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_oir[i]=equantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=equantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=equantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=equantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}

VaRCont_oir
VaRfr1_oir
VaRfr2_oir
CVaRCont_oir
CVaRfr1_oir
CVaRfr2_oir


#VaR Total
VaRTotal_oir=equantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=equantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=equantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=equantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])


print(cbind(VaRTotal_oir,sum(V0_oir), VaRCont_oir, V0_oir))
print(cbind(CVaRTotal_oir,sum(V0_oir), CVaRCont_oir, V0_oir))
print(cbind(VaRTotal_oir,VaRTotalfr1_oir,VaRTotalfr2_oir,VaRTotalfr3_oir))
print(cbind(CVaRTotal_oir,CVaRTotalfr1_oir,CVaRTotalfr2_oir,CVaRTotalfr3_oir))


In [ ]:
%%R
#VaR Total
VaRTotal_oir=wquantile(PGT_oir,p_esc,1-alpha)
CVaRTotal_oir= wcvar(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=wquantile(PGfr1T_oir,p_esc,1-alpha)
CVaRTotalfr1_oir= wcvar(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=wquantile(PGfr2T_oir,p_esc,1-alpha)
CVaRTotalfr2_oir= wcvar(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=wquantile(PGfr3T_oir,p_esc,1-alpha)
CVaRTotalfr3_oir= wcvar(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])


print(cbind(VaRTotal_oir,sum(V0_oir), VaRCont_oir, V0_oir))
print(cbind(CVaRTotal_oir,sum(V0_oir), CVaRCont_oir, V0_oir))
print(cbind(VaRTotal_oir,VaRTotalfr1_oir,VaRTotalfr2_oir,VaRTotalfr3_oir))
print(cbind(CVaRTotal_oir,CVaRTotalfr1_oir,CVaRTotalfr2_oir,CVaRTotalfr3_oir))

##Riesgo de Forwards TdC

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC
#dimensión
m=ncol(plazos_fwd)  #PASO CLAVE
X_s_fwtdc=matrix(0,Ns,n_if[5]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwtdc=matrix(0,Ns,m)
Vfr1_fwtdc=matrix(0,Ns,m)
Vfr2_fwtdc=matrix(0,Ns,m)
Vfr3_fwtdc=matrix(0,Ns,m)
PG_fwtdc=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwtdc=matrix(0,Ns,m)
PGfr2_fwtdc=matrix(0,Ns,m)
PGfr3_fwtdc=matrix(0,Ns,m)

DeltaX_s_fwtdc=DeltaX_s[,sum(n_if[1:4],1):sum(n_if[1:5])]  #PASO CLAVE
x0_fwtdc=X_futtdc[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_fwtdc[i,]=x0_fwtdc*exp(DeltaX_s_fwtdc[i,])
  #PASO CLAVE
  V_fwtdc[i,]=futuroTC(plazos_fwd,X_s_fwtdc[i,1:((n_if[5]-1)/2)],X_s_fwtdc[i,((n_if[5]-1)/2+1):(n_if[5]-1)],X_s_fwtdc[i,(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd     
  #PASO CLAVE
  Vfr1_fwtdc[i,]=futuroTC(plazos_fwd,X_s_fwtdc[i,1:((n_if[5]-1)/2)],x0_fwtdc[((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd
  #PASO CLAVE
  Vfr2_fwtdc[i,]=futuroTC(plazos_fwd,x0_fwtdc[1:((n_if[5]-1)/2)],X_s_fwtdc[i,((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd 
   #PASO CLAVE
  Vfr3_fwtdc[i,]=futuroTC(plazos_fwd,x0_fwtdc[1:((n_if[5]-1)/2)],x0_fwtdc[((n_if[5]-1)/2+1):(n_if[5]-1)],X_s_fwtdc[i,(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd
  PG_fwtdc[i,]=V_fwtdc[i,]-V0_fwtdc
  PGfr1_fwtdc[i,]=Vfr1_fwtdc[i,]-V0_fwtdc
  PGfr2_fwtdc[i,]=Vfr2_fwtdc[i,]-V0_fwtdc
  PGfr3_fwtdc[i,]=Vfr3_fwtdc[i,]-V0_fwtdc
}


#VaR por posición
VaRCont_fwtdc=matrix(0,1,m)
VaRfr1_fwtdc=matrix(0,1,m)
VaRfr2_fwtdc=matrix(0,1,m)
VaRfr3_fwtdc=matrix(0,1,m)
CVaRCont_fwtdc=matrix(0,1,m)
CVaRfr1_fwtdc=matrix(0,1,m)
CVaRfr2_fwtdc=matrix(0,1,m)
CVaRfr3_fwtdc=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_fwtdc[i]=equantile(PG_fwtdc[,i],1-alpha,Ns)
  VaRfr1_fwtdc[i]=equantile(PGfr1_fwtdc[,i],1-alpha,Ns)
  VaRfr2_fwtdc[i]=equantile(PGfr2_fwtdc[,i],1-alpha,Ns)
  VaRfr3_fwtdc[i]=equantile(PGfr3_fwtdc[,i],1-alpha,Ns)
  CVaRfr1_fwtdc[i]= mean(merge(which(PGfr1_fwtdc[,i]<VaRfr1_fwtdc[i]),cbind(seq(1,Ns),PGfr1_fwtdc[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_fwtdc[i]= mean(merge(which(PGfr2_fwtdc[,i]<VaRfr2_fwtdc[i]),cbind(seq(1,Ns),PGfr2_fwtdc[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_fwtdc[i]= mean(merge(which(PGfr3_fwtdc[,i]<VaRfr3_fwtdc[i]),cbind(seq(1,Ns),PGfr3_fwtdc[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_fwtdc[i]= mean(merge(which(PG_fwtdc[,i]<VaRCont_fwtdc[i]),cbind(seq(1,Ns),PG_fwtdc[,i]), by.x=1,by.y=1)[,2])
}
#VaR con alisado 

VaRCont_fwtdcA=matrix(0,1,m)
VaRfr1_fwtdcA=matrix(0,1,m)
VaRfr2_fwtdcA=matrix(0,1,m)
VaRfr3_fwtdcA=matrix(0,1,m)
CVaRCont_fwtdcA=matrix(0,1,m)
CVaRfr1_fwtdcA=matrix(0,1,m)
CVaRfr2_fwtdcA=matrix(0,1,m)
CVaRfr3_fwtdcA=matrix(0,1,m)

for (i in (1:m)) {
  VaRCont_fwtdcA[i]=wquantile(as.matrix(PG_fwtdc[,i]), p_esc[order(p_esc)], 1-alpha)
  VaRfr1_fwtdcA[i]=wquantile(as.matrix(PGfr1_fwtdc[,i]), p_esc[order(p_esc)], 1-alpha)
  VaRfr2_fwtdcA[i]=wquantile(as.matrix(PGfr2_fwtdc[,i]), p_esc[order(p_esc)], 1-alpha)
  VaRfr3_fwtdcA[i]=wquantile(as.matrix(PGfr3_fwtdc[,i]), p_esc[order(p_esc)], 1-alpha)
  CVaRfr1_fwtdcA[i]= wcvar(PGfr1_fwtdc[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
  CVaRfr2_fwtdcA[i]= wcvar(PGfr2_fwtdc[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
  CVaRfr3_fwtdcA[i]= wcvar(PGfr3_fwtdc[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
  CVaRCont_fwtdcA[i]= wcvar(PG_fwtdc[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
}

resulTC<- cbind(VaRfr1_fwtdc, VaRfr2_fwtdc, VaRfr3_fwtdc, VaRCont_fwtdc)
resultTC<-cbind(CVaRfr1_fwtdc, CVaRfr2_fwtdc, CVaRfr3_fwtdc, CVaRCont_fwtdc)
resultadosFtdc <- rbind(resulTC, resultTC)
colnames(resultadosFtdc)<- c("libor", "forward", "tipo de cambio", "Total")
rownames(resultadosFtdc)<- c("VaR", "CVaR")
print(resultadosFtdc)

resulTCA<- cbind(VaRfr1_fwtdcA, VaRfr2_fwtdcA, VaRfr3_fwtdcA, VaRCont_fwtdcA)
resultTCA<-cbind(CVaRfr1_fwtdcA, CVaRfr2_fwtdcA, CVaRfr3_fwtdcA, CVaRCont_fwtdcA)
resultadosFtdcA <- rbind(resulTCA, resultTCA)
colnames(resultadosFtdcA)<- c("libor", "forward", "tipo de cambio", "Total")
rownames(resultadosFtdcA)<- c("VaR CA", "CVaR CA")
print(resultadosFtdcA)

##Riesgo Forward Índice

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS IPC
m=ncol(plazos_fwd_ind)  #PASO CLAVE
X_s_fwind=matrix(0,Ns,n_if[6]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwind=matrix(0,Ns,m)
Vfr1_fwind=matrix(0,Ns,m)
Vfr2_fwind=matrix(0,Ns,m)
Vfr3_fwind=matrix(0,Ns,m)
PG_fwind=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwind=matrix(0,Ns,m)
PGfr2_fwind=matrix(0,Ns,m)
PGfr3_fwind=matrix(0,Ns,m)

DeltaX_s_fwind=DeltaX_s[,sum(n_if[1:5],1):sum(n_if[1:6])]  #PASO CLAVE
x0_fwind=X_futind[1,] #PASO CLAVE

for (i in 1:Ns){
  X_s_fwind[i,]=x0_fwind* exp(DeltaX_s_fwind[i,])
  #PASO CLAVE
  V_fwind[i,]=futuroI(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind     
  Vfr1_fwind[i,]=futuroI(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],x0_fwind[(n_if[6]/3+1):(n_if[6]*2/3)],x0_fwind[(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  #PASO CLAVE
  Vfr2_fwind[i,]=futuroI(plazos_fwd_ind,x0_fwind[1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],x0_fwind[(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
  #PASO CLAVE
  Vfr3_fwind[i,]=futuroI(plazos_fwd_ind,x0_fwind[1:(n_if[6]/3)],x0_fwind[(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  PG_fwind[i,]=V_fwind[i,]-V0_fwind
  PGfr1_fwind[i,]=Vfr1_fwind[i,]-V0_fwind
  PGfr2_fwind[i,]=Vfr2_fwind[i,]-V0_fwind
  PGfr3_fwind[i,]=Vfr3_fwind[i,]-V0_fwind
}


#VaR por posición
VaRCont_fwind=matrix(0,1,m)
VaRfr1_fwind=matrix(0,1,m)
VaRfr2_fwind=matrix(0,1,m)
VaRfr3_fwind=matrix(0,1,m)
CVaRCont_fwind=matrix(0,1,m)
CVaRfr1_fwind=matrix(0,1,m)
CVaRfr2_fwind=matrix(0,1,m)
CVaRfr3_fwind=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_fwind[i]=equantile(PG_fwind[,i],1-alpha,Ns)
  VaRfr1_fwind[i]=equantile(PGfr1_fwind[,i],1-alpha,Ns)
  VaRfr2_fwind[i]=equantile(PGfr2_fwind[,i],1-alpha,Ns)
  VaRfr3_fwind[i]=equantile(PGfr3_fwind[,i],1-alpha,Ns)
  CVaRfr1_fwind[i]= mean(merge(which(PGfr1_fwind[,i]<VaRfr1_fwind[i]),cbind(seq(1,Ns),PGfr1_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_fwind[i]= mean(merge(which(PGfr2_fwind[,i]<VaRfr2_fwind[i]),cbind(seq(1,Ns),PGfr2_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_fwind[i]= mean(merge(which(PGfr3_fwind[,i]<VaRfr3_fwind[i]),cbind(seq(1,Ns),PGfr3_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_fwind[i]= mean(merge(which(PG_fwind[,i]<VaRCont_fwind[i]),cbind(seq(1,Ns),PG_fwind[,i]), by.x=1,by.y=1)[,2])
}
#VaR con alisado 
VaRCont_fwindA=matrix(0,1,m)
VaRfr1_fwindA=matrix(0,1,m)
VaRfr2_fwindA=matrix(0,1,m)
VaRfr3_fwindA=matrix(0,1,m)
CVaRCont_fwindA=matrix(0,1,m)
CVaRfr1_fwindA=matrix(0,1,m)
CVaRfr2_fwindA=matrix(0,1,m)
CVaRfr3_fwindA=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_fwindA[i]=wquantile(as.matrix(PG_fwind[,i]), p_esc[order(p_esc)], 1-alpha)
  VaRfr1_fwindA[i]=wquantile(as.matrix(PGfr1_fwind[,i]), p_esc[order(p_esc)], 1-alpha)
  VaRfr2_fwindA[i]=wquantile(as.matrix(PGfr2_fwind[,i]), p_esc[order(p_esc)], 1-alpha)
  VaRfr3_fwindA[i]=wquantile(as.matrix(PGfr3_fwind[,i]), p_esc[order(p_esc)], 1-alpha)
  CVaRfr1_fwindA[i]= wcvar(PGfr1_fwind[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
  CVaRfr2_fwindA[i]= wcvar(PGfr2_fwind[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
  CVaRfr3_fwindA[i]= wcvar(PGfr3_fwind[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
  CVaRCont_fwindA[i]= wcvar(PG_fwind[,i], as.matrix(p_esc[order(p_esc)]), 1-alpha)
}

resulI<- cbind(VaRfr1_fwind, VaRfr2_fwind, VaRfr3_fwind, VaRCont_fwind)
resultI<-cbind(CVaRfr1_fwind, CVaRfr2_fwind, CVaRfr3_fwind, CVaRCont_fwind)
resultadosFI <- rbind(resulI, resultI)
colnames(resultadosFI)<- c("dividendos", "Pagares", "IPC", "Total")
rownames(resultadosFI)<- c("VaR SA", "CVaR SA")
print(resultadosFI)

resulIA<- cbind(VaRfr1_fwindA, VaRfr2_fwindA, VaRfr3_fwindA, VaRCont_fwindA)
resultIA<-cbind(CVaRfr1_fwindA, CVaRfr2_fwindA, CVaRfr3_fwindA, CVaRCont_fwindA)
resultadosFIA <- rbind(resulIA, resultIA)
colnames(resultadosFIA)<- c("dividendos", "Pagares", "IPC", "Total")
rownames(resultadosFIA)<- c("VaR CA", "CVaR CA")
print(resultadosFIA)


# Riesgo integral

In [ ]:
%%R

#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
PGPort_ACC=PGfr2T_acc_div #+ PGfr3T_fwind #Pérdidas y ganancias
VaRPort_ACC=equantile(PGPort_ACC,1-alpha,Ns) #VaR
CVaRPort_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_ACC)]) #CVaR


#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc
#PGPort_TI=PGT_bd+PGT_sw+PGfr1T_oir +PGfr2T_oir + PGfr1T_fwind +PGfr2T_fwind + PGfr1T_fwtdc + PGfr2T_fwtdc#Pérdidas y ganancias
PGPort_TI=PGT_bd+PGT_sw+PGfr1T_oir +PGfr2T_oir                                                           #Pérdidas y ganancias
VaRPort_TI=equantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc
#PGPort_TDC=PGfr1T_acc_div + PGfr3T_fwtdc  #Pérdidas y ganancias
PGPort_TDC=PGfr1T_acc_div                 #Pérdidas y ganancias
VaRPort_TDC=equantile(PGPort_TDC,1-alpha,Ns) #VaR
CVaRPort_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_TDC)]) #CVaR

#Volatilidad
#1. Sólo aplica la volatilidad de Opciones de tasa de interés

PGPort_VOL=PGfr3T_oir  #Pérdidas y ganancias
VaRPort_VOL=equantile(PGPort_VOL,1-alpha,Ns) #VaR
CVaRPort_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_VOL)]) #CVaR


#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos

PGT_Port=PGPort_ACC+PGPort_TI+PGPort_TDC+PGPort_VOL
VaRTotal_Port=equantile(PGT_Port,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]) #CVaR
print(VaRTotal_Port)
print(CVaRTotal_Port)
print(V0T_port)